In [23]:
import requests
from bs4 import BeautifulSoup

In [24]:

USER = 'RakibulRanak' # will take input from the user instead hardcoded
BASE_URL = 'https://github.com'

In [27]:
url = f'{BASE_URL}/{USER}?tab=repositories'

res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
repo_elements = soup.find_all('h3', {'class': 'wb-break-all'})
repo_elements[0]

<h3 class="wb-break-all">
<a href="/RakibulRanak/llm_engineering" itemprop="name codeRepository">
        llm_engineering</a>
<span></span><span class="Label Label--secondary v-align-middle ml-1 mb-1">Public</span>
</h3>

In [34]:
repo_names = []

for element in repo_elements:
    name = element.find('a').text.strip()
    repo_names.append(name)

In [35]:
repo_names[:5]

['llm_engineering',
 'rakibulranak.github.io',
 'linkedIn_auto_jobs_applier_with_AI',
 'Top-Data-Science-AI-Book-Collection',
 'RakibulRanak']